In [1]:
from src import notebook, datasets, build_models,caluclate_basis, custom_blocks, train, utils
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np 
import argparse
from tqdm import tqdm   
from torchsummary import summary

In [ ]:
custom_args = argparse.Namespace(arch=['64', 'M', '128'], 
                                 batch_norm=False,
                                 bias=False,
                                 classifier_layers=[256,256],
                                 classifier_bias=False,
                                 classifier_dropout=0,
                                 avgpool=True,
                                 avgpool_size=[1,1],  # Adjusted avgpool size to (1, 1)
                                 dataset="mnist",
                                 greyscale=False,
                                 n_classes=10,
                                 epochs=10, 
                                 batch_size=64,
                                 optimizer="Adam",
                                 lr=0.001,
                                 criterion="CrossEntropyLoss",
                                 seed=42
                                 )
# Load ResNet-18 model
def load_resnet18(n_classes):
    model = models.resnet18(pretrained=True)
    # Modify the final fully connected layer to have n_classes output units
    model.fc = nn.Linear(model.fc.in_features, n_classes)
    return model

# get dataloaders
train_loader, val_loader, test_loader = datasets.get_dataloaders(custom_args, logfile=None, summaryfile=None, log=False)
# build model
model = load_resnet18(custom_args.n_classes)
#model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
#model = build_models.build_model_from_args(custom_args)
# train
train_losses, train_accuracies, val_accuracies, test_accuracy = train.train(custom_args, model, train_loader, val_loader, test_loader)
#train.test_model(model, test_loader,device= torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
# log results
print(f"Final Test Accuracy: {test_accuracy}")
print(f"Final Validation Accuracy: {val_accuracies[-1]}")
print(f"Final Training Accuracy: {train_accuracies[-1]}")
print(f"Final Training Loss: {train_losses[-1]}")

C:\Users\adamy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\adamy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 157/157 [00:04<00:00, 38.81it/s] 

Test Accuracy: 5.53%


5.53